In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
data_gdelt = pd.read_csv('/content/drive/MyDrive/GDELT Project/data/processed/Edge weights/gdelt_complete.csv')

In [3]:
data_gdelt.head()

,Unnamed: 0,Actor1CountryCode,Actor2CountryCode,AlternateGoldsteinScale,SQLDATE,Material Cooperation,Verbal Cooperation,Verbal Conflict,Material Conflict,GoldsteinScale,NumArticles,AvgTone,Actor1Geo_Lat,Actor2Geo_Lat,Actor1Geo_Long,Actor2Geo_Long,NumberOfRowsBetweenCountries
0,0,ABW,NLD,3.400000,19791123,0.000000,1.000000,0.000000,0.000000,3.400000,4,5.000000,8.000000,8.000000,-66.000000,-66.000000,1
1,1,AFG,AFG,0.501637,19791231,0.069697,0.651515,0.133333,0.145455,0.107273,1161,5.502025,33.846372,33.764114,63.743229,63.742776,330
2,2,AFG,ARE,1.900000,19790811,0.000000,1.000000,0.000000,0.000000,1.900000,8,6.796117,33.000000,27.311133,65.000000,57.911133,3
3,3,AFG,BEL,2.500000,19791231,0.000000,1.000000,0.000000,0.000000,2.500000,18,4.966140,33.000000,50.833300,65.000000,4.333330,2
4,4,AFG,BGR,3.080000,19791110,0.000000,1.000000,0.000000,0.000000,2.829630,100,5.930335,36.273044,38.396333,51.890452,42.511204,27


In [4]:
data_gdelt.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
TIME_STEPS = set(data_gdelt['SQLDATE'])

In [6]:
TIME_STEPS_LEN = len(TIME_STEPS)

In [7]:
TIME_STEPS_LEN

15813

In [8]:
a1_countries = set(data_gdelt['Actor1CountryCode'])
a2_countries = set(data_gdelt['Actor2CountryCode'])

In [9]:
a1_countries.difference(a2_countries) # all countires in a1 are also in a2

set()

In [10]:
enc = LabelEncoder()
countries = list(a1_countries)
label_encodings = enc.fit_transform(countries)

In [11]:
mapping_dict = {}
mapping_dict['node_ids'] = {}
for c, l in zip(countries, label_encodings):
  mapping_dict['node_ids'][c] = l

In [12]:
mapping_dict['time_periods'] = TIME_STEPS_LEN

In [13]:
TIME_STEPS = sorted(TIME_STEPS)

In [14]:
X_country_location = pd.read_csv('/content/drive/MyDrive/GDELT Project/data/processed/X/country_location_data.csv')

In [15]:
X_daily_exchange_rate = pd.read_csv('/content/drive/MyDrive/GDELT Project/data/processed/X/daily_exchange_rates_USD.csv')

In [16]:
X_annual_GDP = pd.read_csv('/content/drive/MyDrive/GDELT Project/data/processed/X/yearly_GDP_data.csv')

In [17]:
X_annual_HDI = pd.read_csv('/content/drive/MyDrive/GDELT Project/data/processed/X/yearly_HDI.csv')

In [18]:
some = X_annual_HDI[X_annual_HDI['date']==1990]

In [19]:
for i in range(1979, 1990):
  some['date'] = i
  X_annual_HDI = X_annual_HDI.append(some)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Get number of nodes

In [20]:
max(mapping_dict['node_ids'].values())

223

In [21]:
reverse_node_ids = {}
for item in mapping_dict['node_ids'].items():
  reverse_node_ids[str(item[1])] = item[0]

In [22]:
mapping_node_ids_time_steps = mapping_dict.copy()

In [23]:
DATA_VERSION = 1

In [24]:
for i, sqldate in enumerate(TIME_STEPS):
  
  if i % 1000 == 0:
    with open(f'/content/drive/MyDrive/GDELT Project/data/processed/Graph PKL/gdelt_data_{DATA_VERSION}.pkl', 'wb') as f:
      pickle.dump(mapping_dict, f)
    mapping_dict = mapping_node_ids_time_steps.copy()
    DATA_VERSION += 1

  date_view = data_gdelt[data_gdelt['SQLDATE'] == sqldate]
  X = []
  edges = []
  weights = []
  Y = np.zeros((224, 224))
  for row in date_view.iterrows():
    values = row[1]
    source = values['Actor1CountryCode']
    target = values['Actor2CountryCode']
    num_source = mapping_dict['node_ids'][source]
    num_target = mapping_dict['node_ids'][target]
    edges.append([num_source, num_target])
    weights.append([
                    values['Material Cooperation'],	
                    values['Verbal Cooperation'],
                    values['Verbal Conflict'],
                    values['Material Conflict'],
                    values['NumArticles'],
                    values['AvgTone'],
                    values['Actor1Geo_Lat'],
                    values['Actor2Geo_Lat'],
                    values['Actor1Geo_Long'],
                    values['Actor2Geo_Long'],
                    values['NumberOfRowsBetweenCountries']
  ])
    Y[num_source][num_target] = values['AlternateGoldsteinScale']

  for j in range(224):
    node_name = reverse_node_ids[str(j)]
    hdi = X_annual_HDI[X_annual_HDI['date']==int(str(sqldate)[:4])]
    if len(hdi) == 0:
      hdi_val = 0
    elif node_name in hdi.columns:
      hdi_val = hdi[node_name].values[0]
    else:
      hdi_val = 0

    gdp = X_annual_GDP[X_annual_GDP['year']==int(str(sqldate)[:4])]
    if len(gdp) == 0:
      gdp_val = 0
    elif node_name in gdp.columns:
      gdp_val = gdp[node_name].values[0]
    else:
      gdp_val = 0

    exch = X_daily_exchange_rate[X_daily_exchange_rate['date'] == sqldate]
    if len(exch) == 0:
      exch_val = 0
    elif node_name in exch.columns:
      exch_val = exch[node_name].values[0]
    else:
      exch_val = 0

    loc = X_country_location[X_country_location['Country Code'] == node_name]
    if len(loc) == 0:
      min_lat, min_long, max_lat, max_long = 0, 0, 0, 0
    else:
      min_lat, min_long, max_lat, max_long = loc['Min Latitude'].values[0], loc['Min Longitude'].values[0], loc['Max Latitude'].values[0], loc['Max Longitude'].values[0]

    X.append([
      hdi_val,
      gdp_val,
      exch_val,
      min_lat,
      min_long,
      max_lat,
      max_long
    ])
  
  mapping_dict[str(i)] = {}
  mapping_dict[str(i)]['index'] = i
  mapping_dict[str(i)]['edges'] = edges
  mapping_dict[str(i)]['weights'] = weights
  mapping_dict[str(i)]['y'] = Y
  mapping_dict[str(i)]['X'] = X